In [4]:
import requests
from tqdm.notebook import tqdm

In [5]:
headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36'
}

In [6]:
#from selenium import webdriver

In [7]:
#d = webdriver.Firefox()

In [8]:
#d.get('https://www.newspapers.com/api/browse/1//us/new-york/brooklyn/times-union_18390/1931/01/18/main-edition')

In [16]:
#d.get_cookies()

In [17]:
api_root = 'https://www.newspapers.com/api/browse/1/us/'

In [18]:
import json
from pathlib import Path

In [19]:
if not Path('newspapers-api/root.json').exists():
    r = requests.get(f'{api_root}', headers=headers)
    root_json = json.loads(r.text)
    if not Path('newspapers-api').exists():
        Path('newspapers-api').mkdir(parents=True)

    Path('newspapers-api/root.json').write_text(r.text)
    
root_json = json.loads(Path('newspapers-api/root.json').read_text())


In [21]:
# download all states

states = [x.get('name') for x in root_json.get('children')]
for state in states:
    if not Path(f'newspapers-api/{state}').exists():
        Path(f'newspapers-api/{state}').mkdir(parents=True)
        
    if not Path(f'newspapers-api/{state}/root.json').exists():
        print(f'---> downloading {state}')
        if r.status_code == 200:
            r = requests.get(f'{api_root}{state}/', headers=headers)
        else:
            exit()
        Path(f'newspapers-api/{state}/root.json').write_text(r.text)

---> downloading puerto-rico
---> downloading rhode-island
---> downloading south-carolina
---> downloading south-dakota
---> downloading tennessee
---> downloading texas
---> downloading utah
---> downloading vermont
---> downloading virginia
---> downloading washington
---> downloading west-virginia
---> downloading wisconsin
---> downloading wyoming


In [33]:
states = [
    'new-york',
    'new-jersey',
    'california',
    'michigan',
    'indiana',
    'ohio',
    'pennsylvania',
    'massachusetts',
    'kansas',
    'texas',
    'kentucky'
]

In [34]:
# download all cities

download_states = states # ['new-york'] # restrict to states

cities = {x: [y.get('name') for y in json.loads(Path(f'newspapers-api/{x}/root.json').read_text()).get('children')] for x in states}

for state in [x for x in states if x in download_states]:
    for city in tqdm(cities[state], bar_format="{l_bar}{bar}{r_bar}", desc=state):
        if not Path(f'newspapers-api/{state}/{city}').exists():
            Path(f'newspapers-api/{state}/{city}').mkdir(parents=True)
        
        if not Path(f'newspapers-api/{state}/{city}/root.json').exists():
            r = requests.get(f'{api_root}{state}/{city}', headers=headers)
            if r.status_code == 200:
                Path(f'newspapers-api/{state}/{city}/root.json').write_text(r.text)
            else:
                exit()
            
            
            
# download all newspapers per city

download_cities = ['albany']

state_counter = 0
for state, cities in tqdm(cities.items(), bar_format="{l_bar}{bar}{r_bar}", desc="hello"):
    state_counter += 1
    city_counter = 0
    if not state in download_states:
        continue
        
    for city in tqdm(cities, leave=False): # [x for x in cities if x in download_cities]:
        city_counter += 1
        # tqdm.set_description('processing newspapers in ' + city + ', ' + state)
        papers = [y.get('name') for y in json.loads(Path(f'newspapers-api/{state}/{city}/root.json').read_text()).get('children')]
        for paper in tqdm(papers, leave=False):
            search_path = f'{state}/{city}/{paper}'
            if not Path(f'newspapers-api/{search_path}').exists():
                Path(f'newspapers-api/{search_path}').mkdir(parents=True)

            if not Path(f'newspapers-api/{search_path}/root.json').exists():
                print('downloading')
                # tqdm.set_description(f'---> downloading {paper} (city {city_counter}/{len(cities)}, state {state_counter}/{len(states)})')
                r = requests.get(f'{api_root}{search_path}', headers=headers)
                if r.status_code == 200:
                    Path(f'newspapers-api/{search_path}/root.json').write_text(r.text)
                else:
                    exit()
                

In [35]:
# find relevant papers
min_year = 1930
max_year = 1940

In [36]:
all_states = [x['name'] for x in json.loads(Path('newspapers-api/root.json').read_text()).get('children')]
all_cities = {x: [y.get('name') for y in json.loads(Path(f'newspapers-api/{x}/root.json').read_text()).get('children')] for x in states}
newspapers_in_range = {}

for state, cities in all_cities.items():
    print(state)
    for city in cities:
        if Path(f'newspapers-api/{state}/{city}/root.json').exists():
            all_newspapers = [x['name'] for x in json.loads(Path(f'newspapers-api/{state}/{city}/root.json').read_text()).get('children')]
            for newspaper in all_newspapers:
                if Path(f'newspapers-api/{state}/{city}/{newspaper}/root.json').exists():
                    year_range = [int(x['name']) for x in json.loads(Path(f'newspapers-api/{state}/{city}/{newspaper}/root.json').read_text()).get('children')]
                    start_year = min(year_range)
                    end_year = max(year_range)
                    in_min_max = [x for x in year_range if x > min_year and x < max_year]
                    if in_min_max:
                        if not state in newspapers_in_range:
                            newspapers_in_range[state] = {}
                        if not city in newspapers_in_range[state]:
                            newspapers_in_range[state][city] = []
                            
                        newspapers_in_range[state][city].append((newspaper, in_min_max))

new-york
new-jersey
california
michigan
indiana
ohio
pennsylvania
massachusetts
kansas
texas
kentucky


In [ ]:
for state, _ in newspapers_in_range.items():
    print(f'processing {state}')
    for city, newspapers in _.items():
        for newspaper, years in newspapers:
            
            for year in years:
                print(newspaper, year)
                if not Path(f'newspapers-api/{state}/{city}/{newspaper}/{year}/').exists():
                    Path(f'newspapers-api/{state}/{city}/{newspaper}/{year}').mkdir(parents=True)

                if not Path(f'newspapers-api/{state}/{city}/{newspaper}/{year}/root.json').exists():
                    print(f'-----> downloading {newspaper} ({state}/{city})')
                    request_url = f'{api_root}{state}/{city}/{newspaper}/{year}'
                    r = requests.get(request_url, headers=headers)
                    if r.status_code == 200:
                        Path(f'newspapers-api/{state}/{city}/{newspaper}/{year}/root.json').write_text(r.text)
                    else:
                        exit()

processing new-york
the-angola-record_6387 1931
-----> downloading the-angola-record_6387 (new-york/angola)
the-angola-record_6387 1932
-----> downloading the-angola-record_6387 (new-york/angola)
the-angola-record_6387 1933
-----> downloading the-angola-record_6387 (new-york/angola)
the-angola-record_6387 1934
-----> downloading the-angola-record_6387 (new-york/angola)
the-angola-record_6387 1935
-----> downloading the-angola-record_6387 (new-york/angola)
the-angola-record_6387 1936
-----> downloading the-angola-record_6387 (new-york/angola)
the-angola-record_6387 1937
-----> downloading the-angola-record_6387 (new-york/angola)
press-and-sun-bulletin_3799 1931
-----> downloading press-and-sun-bulletin_3799 (new-york/binghamton)
press-and-sun-bulletin_3799 1932
-----> downloading press-and-sun-bulletin_3799 (new-york/binghamton)
press-and-sun-bulletin_3799 1933
-----> downloading press-and-sun-bulletin_3799 (new-york/binghamton)
press-and-sun-bulletin_3799 1934
-----> downloading press-

the-post-star_6162 1933
-----> downloading the-post-star_6162 (new-york/glens-falls)
the-post-star_6162 1934
-----> downloading the-post-star_6162 (new-york/glens-falls)
the-post-star_6162 1935
-----> downloading the-post-star_6162 (new-york/glens-falls)
the-post-star_6162 1936
-----> downloading the-post-star_6162 (new-york/glens-falls)
the-post-star_6162 1937
-----> downloading the-post-star_6162 (new-york/glens-falls)
the-post-star_6162 1938
-----> downloading the-post-star_6162 (new-york/glens-falls)
the-post-star_6162 1939
-----> downloading the-post-star_6162 (new-york/glens-falls)
the-sun-and-the-erie-county-independent_1867 1931
-----> downloading the-sun-and-the-erie-county-independent_1867 (new-york/hamburg)
the-sun-and-the-erie-county-independent_1867 1932
-----> downloading the-sun-and-the-erie-county-independent_1867 (new-york/hamburg)
the-sun-and-the-erie-county-independent_1867 1933
-----> downloading the-sun-and-the-erie-county-independent_1867 (new-york/hamburg)
the-su

the-daily-item_26895 1939
-----> downloading the-daily-item_26895 (new-york/port-chester)
poughkeepsie-eagle-news_3832 1931
-----> downloading poughkeepsie-eagle-news_3832 (new-york/poughkeepsie)
poughkeepsie-eagle-news_3832 1932
-----> downloading poughkeepsie-eagle-news_3832 (new-york/poughkeepsie)
poughkeepsie-eagle-news_3832 1933
-----> downloading poughkeepsie-eagle-news_3832 (new-york/poughkeepsie)
poughkeepsie-eagle-news_3832 1934
-----> downloading poughkeepsie-eagle-news_3832 (new-york/poughkeepsie)
poughkeepsie-eagle-news_3832 1935
-----> downloading poughkeepsie-eagle-news_3832 (new-york/poughkeepsie)
poughkeepsie-eagle-news_3832 1936
-----> downloading poughkeepsie-eagle-news_3832 (new-york/poughkeepsie)
poughkeepsie-eagle-news_3832 1937
-----> downloading poughkeepsie-eagle-news_3832 (new-york/poughkeepsie)
poughkeepsie-eagle-news_3832 1938
-----> downloading poughkeepsie-eagle-news_3832 (new-york/poughkeepsie)
poughkeepsie-eagle-news_3832 1939
-----> downloading poughkeep

the-courier-news_3776 1933
-----> downloading the-courier-news_3776 (new-jersey/bridgewater)
the-courier-news_3776 1934
-----> downloading the-courier-news_3776 (new-jersey/bridgewater)
the-courier-news_3776 1935
-----> downloading the-courier-news_3776 (new-jersey/bridgewater)
the-courier-news_3776 1936
-----> downloading the-courier-news_3776 (new-jersey/bridgewater)
the-courier-news_3776 1937
-----> downloading the-courier-news_3776 (new-jersey/bridgewater)
the-courier-news_3776 1938
-----> downloading the-courier-news_3776 (new-jersey/bridgewater)
the-courier-news_3776 1939
-----> downloading the-courier-news_3776 (new-jersey/bridgewater)
courier-post_3767 1931
-----> downloading courier-post_3767 (new-jersey/camden)
courier-post_3767 1932
-----> downloading courier-post_3767 (new-jersey/camden)
courier-post_3767 1933
-----> downloading courier-post_3767 (new-jersey/camden)
courier-post_3767 1934
-----> downloading courier-post_3767 (new-jersey/camden)
courier-post_3767 1935
----->

keyport-enterprise_8102 1939
-----> downloading keyport-enterprise_8102 (new-jersey/keyport)
keyport-weekly_8096 1931
-----> downloading keyport-weekly_8096 (new-jersey/keyport)
keyport-weekly_8096 1932
-----> downloading keyport-weekly_8096 (new-jersey/keyport)
keyport-weekly_8096 1933
-----> downloading keyport-weekly_8096 (new-jersey/keyport)
keyport-weekly_8096 1934
-----> downloading keyport-weekly_8096 (new-jersey/keyport)
keyport-weekly_8096 1935
-----> downloading keyport-weekly_8096 (new-jersey/keyport)
keyport-weekly_8096 1936
-----> downloading keyport-weekly_8096 (new-jersey/keyport)
keyport-weekly_8096 1937
-----> downloading keyport-weekly_8096 (new-jersey/keyport)
keyport-weekly_8096 1938
-----> downloading keyport-weekly_8096 (new-jersey/keyport)
keyport-weekly_8096 1939
-----> downloading keyport-weekly_8096 (new-jersey/keyport)
the-keyport-weekly-and-the-keyport-enterprise_9263 1931
-----> downloading the-keyport-weekly-and-the-keyport-enterprise_9263 (new-jersey/keyp

...

jackson-county-banner_4585 1938
-----> downloading jackson-county-banner_4585 (indiana/brownstown)
jackson-county-banner_4585 1939
-----> downloading jackson-county-banner_4585 (indiana/brownstown)
national-road-traveler_96 1936
-----> downloading national-road-traveler_96 (indiana/cambridge-city)
the-cambridge-city-tribune_201 1931
-----> downloading the-cambridge-city-tribune_201 (indiana/cambridge-city)
the-cambridge-city-tribune_201 1932
-----> downloading the-cambridge-city-tribune_201 (indiana/cambridge-city)
the-cambridge-city-tribune_201 1933
-----> downloading the-cambridge-city-tribune_201 (indiana/cambridge-city)
the-cambridge-city-tribune_201 1934
-----> downloading the-cambridge-city-tribune_201 (indiana/cambridge-city)
the-cambridge-city-tribune_201 1935
-----> downloading the-cambridge-city-tribune_201 (indiana/cambridge-city)
the-cambridge-city-tribune_201 1936
-----> downloading the-cambridge-city-tribune_201 (indiana/cambridge-city)
the-cambridge-city-tribune_201 1937

the-hancock-democrat_3639 1931
-----> downloading the-hancock-democrat_3639 (indiana/greenfield)
the-hancock-democrat_3639 1932
-----> downloading the-hancock-democrat_3639 (indiana/greenfield)
the-hancock-democrat_3639 1933
-----> downloading the-hancock-democrat_3639 (indiana/greenfield)
the-hancock-democrat_3639 1934
-----> downloading the-hancock-democrat_3639 (indiana/greenfield)
the-hancock-democrat_3639 1935
-----> downloading the-hancock-democrat_3639 (indiana/greenfield)
the-hancock-democrat_3639 1936
-----> downloading the-hancock-democrat_3639 (indiana/greenfield)
the-hancock-democrat_3639 1937
-----> downloading the-hancock-democrat_3639 (indiana/greenfield)
the-hancock-democrat_3639 1938
-----> downloading the-hancock-democrat_3639 (indiana/greenfield)
the-hancock-democrat_3639 1939
-----> downloading the-hancock-democrat_3639 (indiana/greenfield)
greensburg-daily-news_10 1931
-----> downloading greensburg-daily-news_10 (indiana/greensburg)
greensburg-daily-news_10 1932
--

logansport-pharos-tribune_4 1939
-----> downloading logansport-pharos-tribune_4 (indiana/logansport)
the-reporter-times_12485 1932
-----> downloading the-reporter-times_12485 (indiana/martinsville)
the-reporter-times_12485 1936
-----> downloading the-reporter-times_12485 (indiana/martinsville)
the-reporter-times_12485 1937
-----> downloading the-reporter-times_12485 (indiana/martinsville)
the-reporter-times_12485 1938
-----> downloading the-reporter-times_12485 (indiana/martinsville)
the-reporter-times_12485 1939
-----> downloading the-reporter-times_12485 (indiana/martinsville)
mitchell-tribune_18595 1931
-----> downloading mitchell-tribune_18595 (indiana/mitchell)
mitchell-tribune_18595 1932
-----> downloading mitchell-tribune_18595 (indiana/mitchell)
mitchell-tribune_18595 1933
-----> downloading mitchell-tribune_18595 (indiana/mitchell)
mitchell-tribune_18595 1934
-----> downloading mitchell-tribune_18595 (indiana/mitchell)
mitchell-tribune_18595 1935
-----> downloading mitchell-tr

the-richmond-item_4965 1934
-----> downloading the-richmond-item_4965 (indiana/richmond)
the-richmond-item_4965 1935
-----> downloading the-richmond-item_4965 (indiana/richmond)
the-richmond-item_4965 1936
-----> downloading the-richmond-item_4965 (indiana/richmond)
the-richmond-item_4965 1937
-----> downloading the-richmond-item_4965 (indiana/richmond)
the-richmond-item_4965 1938
-----> downloading the-richmond-item_4965 (indiana/richmond)
the-richmond-item_4965 1939
-----> downloading the-richmond-item_4965 (indiana/richmond)
rushville-republican_752 1931
-----> downloading rushville-republican_752 (indiana/rushville)
rushville-republican_752 1932
-----> downloading rushville-republican_752 (indiana/rushville)
rushville-republican_752 1933
-----> downloading rushville-republican_752 (indiana/rushville)
rushville-republican_752 1934
-----> downloading rushville-republican_752 (indiana/rushville)
rushville-republican_752 1935
-----> downloading rushville-republican_752 (indiana/rushvil

vidette-messenger-of-porter-county_6010 1936
-----> downloading vidette-messenger-of-porter-county_6010 (indiana/valparaiso)
vidette-messenger-of-porter-county_6010 1937
-----> downloading vidette-messenger-of-porter-county_6010 (indiana/valparaiso)
vidette-messenger-of-porter-county_6010 1938
-----> downloading vidette-messenger-of-porter-county_6010 (indiana/valparaiso)
vidette-messenger-of-porter-county_6010 1939
-----> downloading vidette-messenger-of-porter-county_6010 (indiana/valparaiso)
the-waterloo-press_975 1932
-----> downloading the-waterloo-press_975 (indiana/waterloo)
the-waterloo-press_975 1934
-----> downloading the-waterloo-press_975 (indiana/waterloo)
the-waterloo-press_975 1935
-----> downloading the-waterloo-press_975 (indiana/waterloo)
the-waterloo-press_975 1936
-----> downloading the-waterloo-press_975 (indiana/waterloo)
the-waterloo-press_975 1937
-----> downloading the-waterloo-press_975 (indiana/waterloo)
the-waterloo-press_975 1938
-----> downloading the-wate